In [1]:
pip install tensorflow scikit-learn pyshark boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 88.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.2 MB/s eta 0:00:00


In [3]:
import pyshark
import pandas as pd

def extract_features_from_pcap(pcap_file):
    capture = pyshark.FileCapture(pcap_file)
    features = []

    for packet in capture:
        try:
            features.append({
                'protocol': packet.highest_layer,
                'length': int(packet.length),
                'src_ip': packet.ip.src if hasattr(packet, 'ip') else None,
                'dst_ip': packet.ip.dst if hasattr(packet, 'ip') else None,
                'src_port': packet[packet.transport_layer].srcport if hasattr(packet, 'transport_layer') else None,
                'dst_port': packet[packet.transport_layer].dstport if hasattr(packet, 'transport_layer') else None,
                'timestamp': float(packet.sniff_timestamp)
            })
        except AttributeError:
            continue

    return pd.DataFrame(features)

data = extract_features_from_pcap('network_traffic.pcap')
data.to_csv('preprocessed_data.csv', index=False)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess the dataset
data = pd.read_csv('preprocessed_data.csv')
data.fillna(0, inplace=True)

# Select features and labels
features = ['length', 'src_port', 'dst_port', 'timestamp']
X = data[features]
y = (data['protocol'] == 'malicious').astype(int)

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a simple neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
import boto3

def upload_alert_to_s3(file_name, bucket_name, object_name):
    """Upload alert logs to an S3 bucket."""
    s3_client = boto3.client('s3')
    s3_client.upload_file(file_name, bucket_name, object_name)

upload_alert_to_s3('alert_log.txt', 'cybersecurity-bucket', 'logs/alert_log.txt')


In [ ]:
def detect_anomalies_live(interface, model, scaler):
    """Detect anomalies in live network traffic."""
    live_capture = pyshark.LiveCapture(interface=interface)

    for packet in live_capture.sniff_continuously():
        try:
            feature = [[
                int(packet.length),
                int(packet[packet.transport_layer].srcport) if hasattr(packet, 'transport_layer') else 0,
                int(packet[packet.transport_layer].dstport) if hasattr(packet, 'transport_layer') else 0,
                float(packet.sniff_timestamp)
            ]]
            feature = scaler.transform(feature)
            prediction = model.predict(feature)

            if prediction > 0.5:
                print(f"Threat detected: {packet.highest_layer}")
                with open('alert_log.txt', 'a') as f:
                    f.write(f"Threat detected: {packet.highest_layer} at {packet.sniff_timestamp}\n")
        except AttributeError:
            continue
